# Setup

In [ ]:
%load_ext sql

In [ ]:
from IPython.core.display import HTML

In [ ]:
%sql $STG_DATABASE_URL

(ipython-sql 0.3.8 doesn't support environment variables, use the one from github)

In [ ]:
!git clone https://github.com/pgexperts/pgx_scripts

# Server

### Version

In [ ]:
%sql SELECT version()

### Extensions

In [ ]:
%sql SELECT * FROM pg_extension;

### Log

In [ ]:
%sql SHOW log_destination  -- not helpful!

http://docs.aws.amazon.com/AmazonRDS/latest/UserGuide/USER_LogAccess.html

## Vacuum

In [ ]:
%sql show autovacuum

### Last vacuum

In [ ]:
result =! psql -H -f pgx_scripts/vacuum/last_autovacuum.sql $DATABASE_URL
HTML(result.s)

## Statistics freshness

In [ ]:
%%sql 
SELECT 
       relname,
       last_vacuum,
       last_autovacuum,
       last_analyze,
       last_autoanalyze
FROM   pg_stat_all_tables
WHERE  schemaname = 'public'

# Tuning-relevant parameters

`shared_buffers` set in Parameter Groups>default.postgres9.4 to `{DBInstanceClassMemory/32768}` 

In [ ]:
%%sql
SELECT name, source, unit,
       pg_size_pretty(setting::numeric * 1024 * 8)  -- b/c 8 kB units
FROM pg_settings
WHERE name IN ('shared_buffers', 'effective_cache_size')
ORDER by 1;



https://www.postgresql.org/docs/9.4/static/runtime-config-resource.html

> If you have a dedicated database server with 1GB or more of RAM, a reasonable starting value for shared_buffers is 25% of the memory in your system.

https://wiki.postgresql.org/wiki/Tuning_Your_PostgreSQL_Server

> Setting effective_cache_size to 1/2 of total memory would be a normal conservative setting, and 3/4 of memory is a more aggressive but still reasonable amount. 

In [ ]:
%%sql
SELECT name, source, unit,
       pg_size_pretty(setting::numeric * 1024 * 1)  -- b/c 1 kB units
FROM pg_settings
WHERE name IN ('work_mem' )
ORDER by 1;

https://wiki.postgresql.org/wiki/Tuning_Your_PostgreSQL_Server

> This size is applied to each and every sort done by each user

# Parameters

## Non-default parameters

In [ ]:
%%sql
SELECT name, source, setting FROM pg_settings WHERE source != 'default'AND source != 'override'ORDER by 2, 1;

### Disk use

In [ ]:
%sql SELECT pg_size_pretty(pg_database_size(current_database()))

#### Biggest tables

[source: PostgreSQL wiki](https://wiki.postgresql.org/wiki/Disk_Usage)

In [ ]:
%%sql
SELECT nspname || '.' || relname AS "relation",
    pg_size_pretty(pg_relation_size(C.oid)) AS "size"
  FROM pg_class C
  LEFT JOIN pg_namespace N ON (N.oid = C.relnamespace)
  WHERE nspname NOT IN ('pg_catalog', 'information_schema')
  ORDER BY pg_relation_size(C.oid) DESC
  LIMIT 20;

# Indexes

## Index size and usage

[Source: PostgreSQL wiki](https://wiki.postgresql.org/wiki/Disk_Usage)

In [ ]:
%%sql 

SELECT
    defn,
    c.reltuples AS num_rows,
    pg_size_pretty(pg_relation_size(quote_ident(t.tablename)::text)) AS table_size,
    pg_size_pretty(pg_relation_size(quote_ident(indexrelname)::text)) AS index_size,
    idx_scan AS number_of_scans,
    idx_tup_read AS tuples_read,
    idx_tup_fetch AS tuples_fetched
FROM pg_tables t
LEFT OUTER JOIN pg_class c ON t.tablename=c.relname
LEFT OUTER JOIN
    ( SELECT c.relname AS ctablename, ipg.relname AS indexname, 
             x.indnatts AS number_of_columns, 
             idx_scan, idx_tup_read, idx_tup_fetch, 
             indexrelname, indisunique,
             pg_get_indexdef(x.indexrelid) AS defn
           FROM pg_index x
           JOIN pg_class c ON c.oid = x.indrelid
           JOIN pg_class ipg ON ipg.oid = x.indexrelid
           JOIN pg_stat_all_indexes psai ON x.indexrelid = psai.indexrelid )
    AS foo
    ON t.tablename = foo.ctablename
WHERE t.schemaname='public'
ORDER BY t.tablename,2;

These stats are since the last stat reset, which occured at:

In [ ]:
%sql select stats_reset from pg_stat_database where datname = 'data_store_api'

## Duplicate indexes

[Source: PostgreSQL wiki](https://wiki.postgresql.org/wiki/Disk_Usage)

In [ ]:
%%sql
SELECT pg_size_pretty(SUM(pg_relation_size(idx))::BIGINT) AS SIZE,
       (array_agg(idx))[1] AS idx1, (array_agg(idx))[2] AS idx2,
       (array_agg(idx))[3] AS idx3, (array_agg(idx))[4] AS idx4
FROM (
    SELECT indexrelid::regclass AS idx, (indrelid::text ||E'\n'|| indclass::text ||E'\n'|| indkey::text ||E'\n'||
                                         COALESCE(indexprs::text,'')||E'\n' || COALESCE(indpred::text,'')) AS KEY
    FROM pg_index) sub
GROUP BY KEY HAVING COUNT(*)>1
ORDER BY SUM(pg_relation_size(idx)) DESC;

## Index bloat

Source: PGXperts

Bloat is fixed with REINDEX.

In [ ]:
result =! psql -H -f pgx_scripts/bloat/index_bloat_check.sql $DATABASE_URL
HTML(result.s)

## Duplicate indexes

In [ ]:
%%sql
SELECT indrelid::regclass AS table_name
     , array_to_string(array_agg(pg_get_indexdef(indexrelid::regclass)), chr(10)) AS indexes
  FROM pg_index
 GROUP BY indrelid
     , indkey
HAVING COUNT(*) > 1;

These "duplicates" seem to be OK

https://www.postgresql.org/docs/9.4/static/indexes-opclass.html

> Note that you should also create an index with the default operator class if you want queries involving ordinary <, <=, >, or >= comparisons to use an index. Such queries cannot use the xxx_pattern_ops operator classes. (Ordinary equality comparisons can use these operator classes, however.) It is possible to create multiple indexes on the same column with different operator classes. If you do use the C locale, you do not need the xxx_pattern_ops operator classes, because an index with the default operator class is usable for pattern-matching queries in the C locale.

In [ ]:
%%sql
SELECT name, source, setting
FROM pg_settings
WHERE name like 'lc_%'
ORDER BY 1;

In [ ]:
%%sql
-- check for matches on only the first column of the index
-- requires some human eyeballing to verify

SELECT indrelid::regclass AS table_name
     , array_to_string(array_agg(pg_get_indexdef(indexrelid)), chr(10)) AS indexes
  FROM pg_index
 GROUP BY indrelid
     , indkey[0]
HAVING COUNT(*) > 1;

## Possibly duplicate indexes

In [ ]:
result =! psql -H -f pgx_scripts/indexes/duplicate_indexes_fuzzy.sql $DATABASE_URL
HTML(result.s)

## Needed indexes

Checks for tables which are getting too much sequential scan activity and might need additional indexing.  Reports in four groups based on table size, number of scans, write activity, and number of existing indexes.

In [ ]:
result =! psql -H -f pgx_scripts/indexes/needed_indexes.sql $DATABASE_URL
HTML(result.s)

### Foreign keys without index

In [ ]:
result =! psql -H -f pgx_scripts/indexes/fk_no_index.sql $DATABASE_URL
HTML(result.s)

# Tables

## Table bloat

In [ ]:
result =! psql -H -f pgx_scripts/bloat/table_bloat_check.sql $DATABASE_URL
HTML(result.s)